# <center>Unsupervised Learing: Text Clustering</center>

References:
* http://brandonrose.org/clustering
* https://www.csee.umbc.edu/~nicholas/clustering/tutorial.pdf
* http://scikit-learn.org/stable/auto_examples/text/document_clustering.html
* https://www-users.cs.umn.edu/~kumar/dmbook/ch8.pdf

## 1. Clustering vs. Categorization
* Clustering (Unsupervised): divide a set of objects into clusters (parts of the set) so that objects in the same cluster are similar to each other, and/or objects in different clusters are dissimilar.
  * Representation of the objects
  * Similarity measure
* Categorization (Supervised): group objects into predetermined categories
  * Representation of the objects
  * A training set

## 2. Why clustering
* Understand conceptually meaningful groups of objects that share common characteristics 
* Provides an abstraction from individual data objects to the clusters in which those data objects reside
* Uses of clustering
  * Summarization
  * Compression
  * Efficiently finding nearest neighbors

## 3. Evaluation of Clustering: What is a good clustering
### 3.1 External Evaluation: 
* External evaluation measures the degree to which predicted clustering labels correspond to actual class labels  
* **Precision** and **Recall**

### 3.2. Internal Evaluation 
<img src='cohension_separation.png' width="60%">
* **Cohension (Intra-cluster similarity)**: how "cohesive" a cluster is, i.e. the average similarity of objects in the same cluster.
   - Radius: $\max{d(x, μ_A)}$ where $μ_A$ is the arithmetic mean of cluster A
   - Average Radius: $\frac{1}{|A|}\sum_{x \in A}{d(x, μ_A)}$
   - Diameter: $\max{d(x, y)}, x \in A, y \in A$
* **Separation (Inter-cluster dissimilarity)**: how "separate" a cluster from another, i.e. the average similarity of all samples in cluster $A$ to all the samples in cluster $B$ may be calcuated
   - Single linkage: $\min(d(x,y)), x \in A,y \in B$
   - Complete linkage: $\max(d(x,y)), x \in A,y \in B$
   - Average distance: $\frac{1}{|A|*|B|}\sum_{x \in A}{\sum_{y \in B}{d(x, y)}}$ 
* Metrics with combined cohension and separation (http://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation)
   - Silhouette Coefficient: $s=\frac{b-a}{\max(a,b)}$, where $a$: the mean distance between a sample and all other points in the same cluster, and $b$: the mean distance between a sample and all other points in the next nearest cluster. $s \in [-1, 1]$.
   - Calinski-Harabaz Index: $s=\frac{b}{a}$ where $a$ is mean within\-cluster separation, and $b$ is the mean between\-cluster separation

## 4. Types of Clusterings
* Hierarchical vs. Patritioning:
  - Partitioning algorithms: flat partition, e.g. K-Means
  - Hierarchical algorithms: hierarchy of clusters, e.g. agglomerative clustering
* Exclusive vs. Overlapping
  - Exclusive: each object is assigned to a single cluster, e.g. K-Means
  - Overlapping (non-exclusive): an object can simultaneously belong to more than one cluster, e.g. LDA

## 5. K-Means
### 5.1. Algorithm outline: Cluster objects into K clusters
<img style="float: left;" src='Kmean1.png'  width='20%'/><img  src='Kmean2.png' width='20%'/>
- Algorithm: 
    1. Select K points as initial centroids 
    2. Repeat until centroids do not change:
        1. Form K clusters by assigning each point to its closest centroid by distance.
        2. Recompute the centroid of each cluster. 
- Distance measure: only Euclidean distance supported in the KMean of SKlean package
   - Warning of "Curse of Dimensionality"

In [ ]:
# Exercise 5.1.1 Load data and generate TF-IDF
# Load datasets (http://qwone.com/~jason/20Newsgroups/)
# For convenience, a subset of the data has been saved into "twenty_news_data.pkl"

import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

data=pickle.load(open("twenty_news_data.pkl","r"))

# Load just two categories: comp.graphics and soc.religion.christian
data=[item for item in data if item[1] in \
      ['comp.graphics','soc.religion.christian']]

# Separate list of tuples to two lists
text,target=zip(*data)
text=list(text)
target=list(target)

# initialize the TfidfVectorizer 
# set min document frequency to 10
# max document frequency to 90%

tfidf_vect = TfidfVectorizer(stop_words="english", min_df=10, max_df=0.9) 

# generate tfidf matrix
dtm= tfidf_vect.fit_transform(text)
print (dtm.shape)

In [ ]:
# Exercise 5.1.2 Clustering using KMean

from nltk.cluster import KMeansClusterer, cosine_distance

# set number of clusters
num_clusters=2
clusterer = KMeansClusterer(num_clusters, cosine_distance, repeats=5)
clusters = clusterer.cluster(dtm.toarray(), assign_clusters=True)

# Note that sklearn has Kmean module
# However, this module uses Euclidean distance
# Due to high dimensionality, it's not used here
# If you like to try, use the following:
# from sklearn.cluster import KMeans
# km = KMeans(n_clusters=num_clusters)
# km.fit(dtm)
# clusters = km.labels_.tolist()

In [ ]:
# Exercise 5.1.3 Interpret each cluster by centroid

# a centroid is the arithemtic mean 
# of all samples in the cluster

# find top words at centroid of each cluster
from sklearn import metrics
import numpy as np

# clusterer.means() contains the centroids
# each row is a cluster, and 
# each column is a feature (word)
# argsort sort the matrix in ascending order 
# and return locations of features before sorting
# [:,::-1] reverse the order

order_centroids = np.array(clusterer.means()).argsort()[:, ::-1] 
# order_centroids has positions of words 
# sorted by their weights in descending order

# get vocabulary: {word: word_index}
voc=tfidf_vect.vocabulary_
# get reversal mapping {word_index: word}
voc_lookup={tfidf_vect.vocabulary_[word]:word \
            for word in tfidf_vect.vocabulary_}

for i in range(num_clusters):
    
    # get words with top 20 tf-idf weight in the centroid
    top_words=[voc_lookup[word_index] \
               for word_index in order_centroids[i, :20]]
    print("Cluster %d: %s " % (i, "; ".join(top_words)))

In [ ]:
# Exercise 5.1.4 Associate cluster to class 
# and calculate performance
import pandas as pd

cluster_dict={0:'comp.graphics', 1:'soc.religion.christian'}
predicted_clusters=[cluster_dict[i] for i in clusters]

print(metrics.classification_report(target, predicted_clusters))

# get confusion matrix
df=pd.DataFrame(zip(target, predicted_clusters), columns=['actual','predict'])
df.groupby(['predict','actual']).size()

In [ ]:
# Exercise 5.1.5 Add "sci.med" documents to the samples and re-do clustering
# Steps:
# 1. add "alt.atheism" to line 13 of Exercise 5.1.1
# 2. change the number of clusters to 3 in line 6 of Exercise 5.1.2
# 3. Re-run Exercise 5.1.1 and Exercise 5.1.2
# 4. Re-run Exercise 5.1.3 to interpret each cluster
# 5. In Exercise 5.1.4. change line 4 based on result of step 4.
# 6. Check the performance metrics of each cluster

## 5.2. Visualization of High-Dimension Data
- High-dimension data can be visualized through dimension reduction techniques
- **Multidimensional scaling (MDS)**: seeks a low-dimensional representation of the data which reserves the  the distances between samples in the original high-dimensional space.
    - MDS attempts to model similarity or dissimilarity data as distances in a geometric space
    - Each sample is converted to coordinates in the geometric space so that the distance between each pair of samples is still reserved

In [ ]:
# Exercise 5.2.1 Convert tf-idf matrix to 2-dimensional coordinates

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import MDS

# distance between any pair of documents
dist = 1 - cosine_similarity(dtm)

# represent data to 2-dimension space
# compute coordinates using precomputed distance 
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
pos = mds.fit_transform(dist)

print(pos[0:5])

In [ ]:
# Exercise 5.2.2 Visualize document simiarity using matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# get color sets. Each cluster will be shown in a different color
colors = ["red", "green", "blue"]
groups = ['comp.graphics', 'soc.religion.christian']

# set up plot
fig = plt.figure(figsize=(16, 6))
# have two subplots horizontally placed
# plot the clusters
ax = fig.add_subplot(1, 2, 1)

# loop through each cluster to plot
for idx, g in enumerate(groups):
    # get coordinates of each cluster
    X=np.array([pos[i] for i in range(dtm.shape[0]) \
                if cluster_dict[clusters[i]]==g])
    ax.scatter(X[:,0], X[:,1], alpha=0.8, \
               c=colors[idx], edgecolors='none', s=30, label=g)
plt.legend()
plt.title('Document Clusters')

# plot the actual categories
ax = fig.add_subplot(1, 2, 2)

# loop through each actual category to plot
for idx, g in enumerate(groups):
    # get coordinates of each category
    X=np.array([pos[i] for i in range(dtm.shape[0]) \
                if target[i]==g])
    ax.scatter(X[:,0], X[:,1], alpha=0.8, \
               c=colors[idx], edgecolors='none', s=30, label=g)    
plt.title('Actual Document Category')
plt.legend()

plt.show()



In [ ]:
# Exercise 5.2.3 Visualize document simiarity using brunel
import pandas as pd
import brunel

# define a dataframe for positions
df=pd.DataFrame(pos, columns=["x","y"])
# add cluster column
df['cluster']=[cluster_dict[i] for i in clusters]
# add actual category column
df['category']=target
#df.head()

%brunel data('df') x(x) y(y) color(cluster) title("Document Clusters") :: width=600, height=500


In [ ]:
%brunel data('df') x(x) y(y) color(category) title("Document Category") :: width=800, height=500

### 5.3. How to pick *K*, the number of clusters?
- Usually, we don't know how many clusters in the data set
- Best **K** can be selected to optimize peformance metrics
  - e.g. Calinski Harabaz score, f1 etc.
  - The best **K** is the one with the best performance metric

In [ ]:
# Exercise 5.3.1. Find the best number of clusters

from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

# distance between any pair of documents

%matplotlib inline

# Since calinski_harabaz_score is calculated based on 
# Euclidean distance, we need to reduce the dimension
# set min_df can reduce the dimension
# However this may distort the distance
tfidf_vect = TfidfVectorizer(stop_words="english", min_df=0.05, max_df=0.9) 

# generate tfidf matrix
dtm1= tfidf_vect.fit_transform(text)
print(dtm1.shape)

# set K from 2 to 9 to see which one has the best score
s=[]
z=dtm.todense()
for k in range(2,10):
    # Multiple round of KMeans with different random seeds may be needed for stability
    km = KMeans(n_clusters=k)
    km.fit(dtm1)
    clusters = km.labels_.tolist()
    # calculate calinski_harabaz metric
    # only dense arrays are allows. 
    # todense() convert sparse arrays to dense arrays
    m=metrics.calinski_harabaz_score(dtm1.toarray(), clusters)
    s.append((k,m))

In [ ]:
# plot Calinski Harabaz score against the number of clusters
# the best K is usually the one around the peak

s=np.array(s)

plt.plot(s[:,0], s[:,1], '-')
plt.xlabel('Number of Clusters')
plt.ylabel('Calinski Harabaz Score')
plt.title('Find the Best K')
plt.show()

## 6. Agglomerative Hierarchical Clustering
* Steps:
    - Starting with each point as a singleton cluster
    - Repeatedly merging the two closest clusters until a single, all-encompassing cluster remains
* The clustering result is represented as a dendrogram
<img src='dendrogram.png' width="30%">

In [ ]:
# Exercise 6.1 Hierarchical clustering

from scipy.cluster.hierarchy import ward, dendrogram
import matplotlib.pyplot as plt
%matplotlib inline 

# create a linkage matrix using ward's method
# the linkage matrix contains the dendragram data
# in ward methods, the distance between two clusters
# is measured as the sum of distance 
# from each sample to the cluster center 

linkage_matrix = ward(dist)

# plot the dendrogram
fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(linkage_matrix, orientation="right");

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout


In [ ]:
# Exercise 6.2 Cut the dendrogram to get clusters

from scipy import cluster
clusters = cluster.hierarchy.cut_tree(linkage_matrix,n_clusters=3)
clusters=clusters.flatten().tolist()

# get documents from each clusters and examine the clusters
cluster0=[text[i] for i in range(len(text)) if clusters[i]==0]
cluster1=[text[i] for i in range(len(text)) if clusters[i]==1]
print (cluster0[0])
print (cluster1[0])

In [ ]:
# Exercise 6.3 Exam a few samples from each cluster to interpret the cluster

cluster_dict={0:'comp.graphics', 1:'soc.religion.christian'}
predicted_clusters=[cluster_dict[i] for i in clusters]

print(metrics.classification_report\
      (target, predicted_clusters, target_names=sorted(cluster_dict.values())))